In [ ]:
import os
import sys  
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define the path to the src directory
src_path = os.path.abspath('../src')

# Check if the src directory exists
if src_path not in sys.path:
    # Append src directory to the Python path
    sys.path.append(src_path)

from data_processing import valid_loader, train_loader, test_loader, labels
from helper import train, evaluate, test, visualise_loss, visualise_all_loss

In [ ]:
num_classes = len(labels)

torch.manual_seed(43)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define Dense Layer
class _DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(_DenseLayer, self).__init__()
        self.norm1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, growth_rate*4, kernel_size=1, stride=1, bias=False)
        self.norm2 = nn.BatchNorm2d(growth_rate*4)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(growth_rate*4, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)
        self.classifier = nn.Linear(growth_rate, len(labels))

    def forward(self, x):
        out = self.conv1(self.relu1(self.norm1(x)))
        out = self.conv2(self.relu2(self.norm2(out)))
        out = torch.cat([x, out], 1)
        return out

In [ ]:
# Define Dense Block
class _DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate):
        super(_DenseBlock, self).__init__()
        self.layers = nn.ModuleList([_DenseLayer(in_channels + i * growth_rate, growth_rate) for i in range(num_layers)])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
# Define Transition Block
class _Transition(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(_Transition, self).__init__()
        self.norm = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        out = self.pool(self.conv(self.relu(self.norm(x))))
        return out

In [ ]:
# Define Custom DenseNet model
class CustomDenseNet(nn.Module):
    def __init__(self):
        super(CustomDenseNet, self).__init__()
        self.conv0 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False).to(torch.float32)
        self.norm0 = nn.BatchNorm2d(64)
        self.relu0 = nn.ReLU(inplace=True)
        self.pool0 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.denseblock1 = _DenseBlock(num_layers=3, in_channels=64, growth_rate=4)  # Reduced growth rate
        self.transition1 = _Transition(in_channels=76, out_channels=128)
        self.denseblock2 = _DenseBlock(num_layers=3, in_channels=128, growth_rate=4)  # Added second dense block
        self.norm5 = nn.BatchNorm2d(140, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)  # Adjusted to the output size of the last dense block
        self.pool1 = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.linear = nn.Linear(in_features=(140), out_features=15, bias=True)  # Adjusted input features

    def forward(self, x):
        x = self.conv0(x)
        x = self.norm0(x)
        x = self.relu0(x)
        out = self.pool0(x)
        out = self.denseblock1(out)
        out = self.transition1(out)
        out = self.denseblock2(out)
        out = F.relu(self.norm5(out))
        out = self.pool1(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        out = F.sigmoid(out)
        return out
    
# Create an instance of Custom DenseNet
custom_densenet = CustomDenseNet()
custom_densenet.to(device)

# Print the model architecture
print(custom_densenet)

In [ ]:
# Clear CUDA memory
torch.cuda.empty_cache()

# Paths for saving
save_dir = "../models/densenet_variation_15e/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Define Params
criterion = nn.BCELoss()
num_epochs = 15
learning_rates = [0.001, 0.0005]

In [16]:
# Track losses for visualization
train_losses_dict = {}
valid_losses_dict = {}

# Iterate over different learning rates
for lr in learning_rates:
    optimizer = optim.Adam(custom_densenet.parameters(), lr=lr)

    best_valid_loss = float('inf')
    train_losses = []  
    valid_losses = []  
    
    for epoch in range(num_epochs):

        avg_train_loss = train(custom_densenet, train_loader, optimizer, criterion, device)
        train_losses.append(avg_train_loss)
        
        valid_loss = evaluate(custom_densenet, valid_loader, criterion, device)
        valid_losses.append(valid_loss)
        
        # Print validation loss
        print(f'Learning Rate: {lr}, Epoch: {epoch+1}, Test Loss: {avg_train_loss:.4f}, Validation Loss: {valid_loss:.4f}')
        
        # Save the best model if validation loss improves
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(custom_densenet.state_dict(), os.path.join(save_dir, f'best_model_lr_{lr}.pt'))

    # Store losses for visualization
    train_losses_dict[lr] = train_losses
    valid_losses_dict[lr] = valid_losses

# Save losses dictionaries for visualization later
torch.save(train_losses_dict, os.path.join(save_dir, 'train_losses.pt'))
torch.save(valid_losses_dict, os.path.join(save_dir, 'valid_losses.pt'))

In [ ]:
# Evaluate Model
for lr in learning_rates:
    model_path = os.path.join(save_dir, f'best_model_lr_{lr}.pt')  # Change the filename accordingly
    custom_densenet.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    metrics, loss = test(custom_densenet, test_loader, criterion, device)
    print(metrics, loss)

In [ ]:
# Plot Train Loss Graph
model_path = os.path.join(save_dir, f'train_losses.pt')  # Change the filename accordingly
visualise_loss(model_path)

In [ ]:
# Plot Validation Loss Graph
model_path = os.path.join(save_dir, f'valid_losses.pt')  # Change the filename accordingly
visualise_loss(model_path)

In [ ]:
# Plot Train Loss and Validation Loss on the same graph
train_model_path = os.path.join(save_dir, f'train_losses.pt')
valid_model_path = os.path.join(save_dir, f'valid_losses.pt')

visualise_all_loss(train_model_path, valid_model_path)